In [ ]:
from ultralytics import YOLO  

In [ ]:
# Create a new YOLO model from scratch
model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

Model Training 

In [ ]:
model.train(data='config.yaml', epochs=300, imgsz=416,device=[0]) #training the model with the yaml file with 150 epochs and image size of 640

In [ ]:
# Load a model
#since model training gets interrupted had to resume the model from it last pt file

model = YOLO('./runs/detect/train17/weights/last.pt')  # load a partially trained model


model.train(resume=True) # Resume training


In [ ]:
model = YOLO('./runs/detect/train18/weights/best.pt')  # load a custom model

# Validate the model

metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

In [ ]:
import torch.onnx
model=YOLO('./runs/detect/train7/weights/best.pt')
model.export(format='onnx') # exporting the mode as onnx

In [ ]:
model=YOLO('./runs/detect/train5/weights/best.pt') # using the best pt file for predicting the results as parameters
results=model(['./Predict/DJI_0268.jpg'])


for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs 

In [ ]:
model = YOLO('./runs/detect/train7/weights/best.pt')
results = model('', save=True)  #finding the predicted plastics for individual images .
                                
                                
#place the path of the image in double quotes and hence results would be stored

Finding OUTPUT Images for predicted values

In [ ]:
image_urls = [                      #looping the images just by copying pasting the image-paths and outputs for the no of images would be stored in same folder
    './Predict/DJI_0023.jpg',
    './Predict/DJI_0080.jpg',
    './Predict/DJI_0088.jpg',
    './Predict/DJI_0185.jpg',
    './Predict/DJI_0205.jpg',
    './Predict/DJI_0255.jpg',
    './Predict/DJI_0268.jpg',
    './Predict/DJI_0416.jpg',
    './Predict/DJI_0487.jpg',
    './Predict/DJI_0502.jpg',
]
for url in image_urls:
    results = model(url, save=True)

Getting the GEO-Location URL

In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS

# Function to format the coordinates to degrees
def convert_to_degrees(value):
    """
    Helper function to convert the GPS coordinates to decimal degrees format.
    """
  
    d = value[0]
    m = value[1]
    s = value[2]

    return d + (m / 60.0) + (s / 3600.0)

#Function to extract geotag from image metadata
def get_geotag_label(image_path):
    # Open the image file
    image = Image.open(image_path)

    # Check if the image has EXIF data
    if hasattr(image, '_getexif'):
        exif_data = image._getexif()

        # Iterate over all EXIF tags
        for tag, value in exif_data.items():
            tag_name = TAGS.get(tag, tag)

            # Check if the tag corresponds to geotagging
            if tag_name == 'GPSInfo':
                # Extract latitude and longitude information
                lat = convert_to_degrees(value[2])
                lon = convert_to_degrees(value[4])
                lat_ref = value[1]
                lon_ref = value[3]

                # Determine the hemisphere (N/S or E/W)
                if lat_ref == 'S':
                    lat = -lat
                if lon_ref == 'W':
                    lon = -lon

                # Return the geotagged label
                return f"Latitude: {lat}, Longitude: {lon}"

    # Return None if no geotag label found
    return None


# this function splits the string containing longitude and latitude into two variables latitude and longitude
def split_geotag_label(geotag_label):
    # Remove the prefix "Latitude: " and extract the latitude value
    latitude_start_index = geotag_label.find(":") + 1
    latitude_end_index = geotag_label.find(",") 
    latitude = geotag_label[latitude_start_index:latitude_end_index].strip()

    # Remove the prefix "Longitude: " and extract the longitude value
    longitude_start_index = geotag_label.find("Longitude:") + len("Longitude:") + 1
    longitude = geotag_label[longitude_start_index:].strip()

    return latitude, longitude




def get_google_maps_link(latitude, longitude):
    link = f"https://www.google.com/maps/search/?api=1&query={latitude},{longitude}"
    return link


Find Geo-Location URL for single image 

In [ ]:
img_path=''   # copy past the complete image path into the single quotes

g_label=get_geotag_label(img_path)
latitude, longitude = split_geotag_label(g_label)
url=get_google_maps_link(latitude, longitude)
print("Image"+".jpg URL: ", url) # this line would print the geo location url of the picture

Finding Geo-Location url for Predict Dataset

In [ ]:
for i in image_urls:
    img_path=str(i)   # the image path url is taken from the above loop (used for find the predict values) and sending it as input and hence 
                      #  the final line gives us the URL link 
    
    g_label=get_geotag_label(img_path)
    latitude, longitude = split_geotag_label(g_label)
    url=get_google_maps_link(latitude, longitude)
    print("Image",str(i)+": ", url)  